In [27]:
import pandas as pd
import json

In [28]:
netflix = pd.read_csv("netflix_cleaned.csv")
netflix.head(2)

,Unnamed: 0,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,month_added,month_name_added,year_added,listed_in_1,listed_in_2,listed_in_3
0,0,Movie,Dick Johnson Is Dead,Kirsten Johnson,['Data not available'],United States,2021-09-25,2020,PG-13,90 min,['Documentaries'],"As her father nears the end of his life, filmm...",9,September,2021,Documentaries,0,0
1,1,TV Show,Blood & Water,Data not available,"['Ama Qamata', 'Khosi Ngema', 'Gail Mabalane',...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"['International TV Shows', 'TV Dramas', 'TV My...","After crossing paths at a party, a Cape Town t...",9,September,2021,International TV Shows,TV Dramas,TV Mysteries


# NETFLIX CONTENTS INFO

In [29]:
netflix_wordcloud = netflix[['type', 'listed_in_1', 'rating', 'title']]
netflix_wordcloud.head(3)

,type,listed_in_1,rating,title
0,Movie,Documentaries,PG-13,Dick Johnson Is Dead
1,TV Show,International TV Shows,TV-MA,Blood & Water
2,TV Show,Crime TV Shows,TV-MA,Ganglands


In [30]:
def pd_to_dict(df):
    yield {
        "name": "netflix", 
        "children": list(split_type(df))
    }

def split_type(df):
    for (type), df_type_grouped in df.groupby(["type"]):
        yield {
            "name": type,
            "children": list(split_categories(df_type_grouped))
                
        }

def split_categories(df_type_grouped):
    for (category), df_listed_grouped in df_type_grouped.groupby(["listed_in_1"]):
        yield {
            "name": category,
            "children": list(split_ratings(df_listed_grouped)),
        }

def split_ratings(df_listed_grouped):
    for (rating), df_ratings_grouped in df_listed_grouped.groupby(["rating"]):
        yield {
            "name": rating,
            "children": list(split_titles(df_ratings_grouped)),
        }

def split_titles(df_ratings_grouped):
    for row in df_ratings_grouped.itertuples():
        yield {
            "name": row.title,
            "value": 1
        }

transactions_list = list(pd_to_dict(netflix_wordcloud))
transactions_dict = {}
for sub_dict in transactions_list:
    transactions_dict.update(sub_dict)
#print(json.dumps(transactions_dict, indent=4))


C:\Users\marco\AppData\Local\Temp\ipykernel_8816\3915687807.py:8: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for (type), df_type_grouped in df.groupby(["type"]):
C:\Users\marco\AppData\Local\Temp\ipykernel_8816\3915687807.py:16: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for (category), df_listed_grouped in df_type_grouped.groupby(["listed_in_1"]):
C:\Users\marco\AppData\Local\Temp\ipykernel_8816\3915687807.py:23: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for (

In [31]:
json_object = json.dumps(transactions_dict)
with open("../Dashboard/Components/data/netflix_sunburst_data.json", "w") as outfile:
    outfile.write(json_object)